In [1]:
import os
import ast
import mongoengine as db
import pandas as pd
from blob_schema import Blob
from tracking_id_schema import Tracking_id
from bson import ObjectId
#from store_schema import Store
from zone_schema import Zone
from video_schema import Video
from cameras_schema import Camera #TODO 
import re
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())
import random
import datetime
from faker import Faker
fake = Faker()

In [68]:
df = pd.read_feather('13_3-1_processed.feather')
df['time_stamp'].iloc[0]

Timestamp('2024-02-25 22:15:01.359274')

In [63]:
df2 = pd.read_csv('16_1-2_processed_updated-2.csv')
df2["time_stamp"] = df2["elapsed_time"]
df2.drop("elapsed_time",axis=1)

,frame_number,process_time,label,pose_bbox,pose_track_ids,fps,image_path,0_department,1_zone,2_zone,3_ground,4_department,time_stamp
0,0,0 days 00:00:00.096962,[1],"[array([ 99, 287, 348, 575])]",[1],25,./results/cctv-seg-1//16_1/16_1-2_processed/im...,NaN,NaN,NaN,NaN,NaN,0 days 00:00:00.259130
1,5,0 days 00:00:00.001963,[1],"[array([ 99, 287, 348, 575])]",[1],25,./results/cctv-seg-1//16_1/16_1-2_processed/im...,NaN,NaN,NaN,NaN,NaN,0 days 00:00:01.673071
2,10,0 days 00:00:00.000531,[1],"[array([ 99, 287, 348, 575])]",[1],25,./results/cctv-seg-1//16_1/16_1-2_processed/im...,NaN,NaN,NaN,NaN,NaN,0 days 00:00:01.849190
3,15,0 days 00:00:00.000529,[1],"[array([ 99, 287, 348, 575])]",[1],25,./results/cctv-seg-1//16_1/16_1-2_processed/im...,NaN,NaN,NaN,NaN,NaN,0 days 00:00:02.098461
4,20,0 days 00:00:00.000568,[1],"[array([ 99, 287, 348, 575])]",[1],25,./results/cctv-seg-1//16_1/16_1-2_processed/im...,NaN,NaN,NaN,NaN,NaN,0 days 00:00:02.252406
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4494,22475,0 days 00:00:00.000543,[0 0 0 0 0 1 1 0 0],"[array([349, 132, 452, 433]) array([ 0, 50, ...",[ 59 80 144 147 145 102 107 108 146],25,./results/cctv-seg-1//16_1/16_1-2_processed/im...,"{25: {'label': 0, 'start_time': 185.6, 'end_ti...","{1: {'label': 1, 'start_time': 14.0, 'end_time...","{25: {'label': 0, 'start_time': 185.6, 'end_ti...","{1: {'label': 1, 'start_time': 4.2, 'end_time'...","{9: {'label': 0, 'start_time': 90.6, 'end_time...",0 days 00:25:54.681743
4495,22480,0 days 00:00:00.000585,[0 0 0 0 0 1 1 0 0],"[array([349, 76, 464, 434]) array([ 0, 50, ...",[ 59 80 144 147 145 102 107 108 146],25,./results/cctv-seg-1//16_1/16_1-2_processed/im...,"{25: {'label': 0, 'start_time': 185.6, 'end_ti...","{1: {'label': 1, 'start_time': 14.0, 'end_time...","{25: {'label': 0, 'start_time': 185.6, 'end_ti...","{1: {'label': 1, 'start_time': 4.2, 'end_time'...","{9: {'label': 0, 'start_time': 90.6, 'end_time...",0 days 00:25:55.237841
4496,22485,0 days 00:00:00.000522,[0 0 0 0 0 1 1 0],"[array([349, 57, 466, 434]) array([ 0, 50, ...",[ 59 80 144 147 145 102 107 108],25,./results/cctv-seg-1//16_1/16_1-2_processed/im...,"{25: {'label': 0, 'start_time': 185.6, 'end_ti...","{1: {'label': 1, 'start_time': 14.0, 'end_time...","{25: {'label': 0, 'start_time': 185.6, 'end_ti...","{1: {'label': 1, 'start_time': 4.2, 'end_time'...","{9: {'label': 0, 'start_time': 90.6, 'end_time...",0 days 00:25:55.496477
4497,22490,0 days 00:00:00.000588,[0 0 0 0 0 1 1 0 1],"[array([349, 48, 467, 434]) array([ 0, 49, ...",[ 59 80 144 147 145 102 107 108 146],25,./results/cctv-seg-1//16_1/16_1-2_processed/im...,"{25: {'label': 0, 'start_time': 185.6, 'end_ti...","{1: {'label': 1, 'start_time': 14.0, 'end_time...","{25: {'label': 0, 'start_time': 185.6, 'end_ti...","{1: {'label': 1, 'start_time': 4.2, 'end_time'...","{9: {'label': 0, 'start_time': 90.6, 'end_time...",0 days 00:25:55.892849


In [58]:
all_ids = []

for index, row in df2.iterrows():
    zone_value = row["1_zone"]
    if not pd.isna(zone_value):
        zone_dict = ast.literal_eval(zone_value)
        
        current_ids = zone_dict.get("current_ids", [])
        
        all_ids.extend(current_ids)

zone1_ids = list(set(all_ids))
for index, row in df2.iterrows():
    zone_value = row["2_zone"]
    if not pd.isna(zone_value):
        zone_dict = ast.literal_eval(zone_value)
        
        current_ids = zone_dict.get("current_ids", [])
        
        all_ids.extend(current_ids)

zone2_ids = list(set(all_ids))
for index, row in df2.iterrows():
    zone_value = row["3_ground"]
    if not pd.isna(zone_value):
        zone_dict = ast.literal_eval(zone_value)
        
        current_ids = zone_dict.get("current_ids", [])
        
        all_ids.extend(current_ids)

ground_ids = list(set(all_ids))
for index, row in df2.iterrows():
    zone_value = row["4_department"]
    if not pd.isna(zone_value):
        zone_dict = ast.literal_eval(zone_value)
        
        current_ids = zone_dict.get("current_ids", [])
        
        all_ids.extend(current_ids)

department_ids = list(set(all_ids))

In [55]:
all_ids = []

for index, row in df2.iterrows():
    zone_value = row["2_zone"]
    if not pd.isna(zone_value):
        zone_dict = ast.literal_eval(zone_value)
        
        current_ids = zone_dict.get("current_ids", [])
        
        all_ids.extend(current_ids)

zone2_ids = list(set(all_ids))

In [56]:
all_ids = []

for index, row in df2.iterrows():
    zone_value = row["3_ground"]
    if not pd.isna(zone_value):
        zone_dict = ast.literal_eval(zone_value)
        
        current_ids = zone_dict.get("current_ids", [])
        
        all_ids.extend(current_ids)

ground_ids = list(set(all_ids))

In [57]:
all_ids = []

for index, row in df2.iterrows():
    zone_value = row["4_department"]
    if not pd.isna(zone_value):
        zone_dict = ast.literal_eval(zone_value)
        
        current_ids = zone_dict.get("current_ids", [])
        
        all_ids.extend(current_ids)

department_ids = list(set(all_ids))

In [3]:
def connect_to_mongo():
    mongo_url = os.environ.get('MONGO_APP_URL')
    db.connect(host=mongo_url)
    print(mongo_url)

In [7]:
pose_track_ids_lists=df["pose_track_ids"]

In [8]:
flat_pose_track_ids = [track_id for sublist in pose_track_ids_lists for track_id in sublist]

unique_track_ids = list(set(flat_pose_track_ids))

In [23]:
def filling_tracking_ids(num_records):
    for i in range(num_records):
        pose_track_ids = unique_track_ids[i]
        labels = list(row['label'])
        person_class = [element for element in labels if element.isdigit()]
        age = random.randint(18, 70)  
        gender = random.choice(['Male', 'Female'])
        upper_body_clothes = random.choice(['Red', 'Blue', 'Green', 'Yellow'])
        lower_body_clothes = random.choice(['Black', 'White', 'Gray'])
        blob_refs = [] 
        store_id = None  
        group = random.choices(['1', '2', '3'], k=random.randint(1, 3))  

        tracking_id = Tracking_id(
            pose_track_ids=pose_track_ids,
            person_class=person_class,
            age=age,
            gender=gender,
            upper_body_clothes=upper_body_clothes,
            lower_body_clothes=lower_body_clothes,
            blob_refs=blob_refs,
            store_id=store_id,
            group=group
        )
        tracking_id.save()

In [ ]:
def filling_blob():
    tracking_id_dict = {}
    '''
    frame_number = db.IntField(required=True)
    time_stamp = db.FloatField(required=True)
    camera_id = db.StringField(required=True)
    pose_bbox = db.ListField(db.IntField())
    fps = db.FloatField(required=True)
    image_path = db.StringField(required=True)
    trackingid_ref = db.ReferenceField('Tracking_id')
    zone_id = db.ReferenceField('Zone')

    '''

    for index,row in df2[2:-1].iterrows():
        frame_no =int( row['frame_number'])
        pose = row['pose_bbox']
        pose_bbox = [list(map(int, re.findall(r'\d+', item))) for item in pose.split(')\n') if item]
        time_stamp = row['time_stamp']
        fps = row['fps']
        image_path = row['image_path']
        zone_id = row['
        print(pose_track_id)

        
    
        for index,value in enumerate(pose_track_id):

            blob = Blob(
                pose_track_ids = value,
                frame_number=frame_no,
                elapsed_time=elaps_time,
                time_stamp = time_stamp,
                pose_bbox=pose_bbox[index],
                fps=fps,
                image_path = image_path,
            ).save()
            track_id = Tracking_id(

            )
            print("blob data saved")
            if value not in tracking_id_dict:
                tracking_id = Tracking_id(
                    pose_track_ids=value,
                    person_class=labels[index],
                    age=45,
                    gender='male',
                    upper_body_clothes='green',
                    lower_body_clothes='white',
                ).save()
                tracking_id_dict[value] = tracking_id
                print("trackingid data saved")
            blob.trackingid_ref = tracking_id_dict[value]
            blob.save()
            tracking_id_dict[value].blob_refs.append(blob.id)
            tracking_id_dict[value].save()

In [13]:
def populate_camera_data():
    dummy_cameras = [
        {
            "store_id": "store_id_1",
            "camera_id": "camera_001",
            "blob": "blob_id_1",  # Assuming you have blob IDs in your database
            "zones": {"zone1": "description1", "zone2": "description2"},
            "analysis_timestamp": datetime.datetime.utcnow(),
            "neo_object": [
                {"id": 1, "coordinates": {"x": 19.09, "y": 25.55}},
                {"id": 2, "coordinates": {"x": 21.35, "y": 28.75}},
            ],
            "created_at": datetime.datetime.datetime.now(),
            "update_at": datetime.datetime.utcnow()
        },
        {
            "store_id": "store_id_2",
            "camera_id": "camera_002",
            "blob": "blob_id_2",  # Assuming you have blob IDs in your database
            "zones": {"zone3": "description3", "zone4": "description4"},
            "analysis_timestamp": datetime.datetime.utcnow(),
            "neo_object": [
                {"id": 3, "coordinates": {"x": 15.20, "y": 30.10}},
                {"id": 4, "coordinates": {"x": 18.75, "y": 22.90}},
            ],
            "created_at": datetime.datetime.utcnow(),
            "update_at": datetime.datetime.utcnow()
        },
    ]

    for camera_data in dummy_cameras:
        camera = Camera(
            store_id=camera_data["store_id"],
            camera_id=camera_data["camera_id"],
            blob=camera_data["blob"],
            zones=camera_data["zones"],
            analysis_timestamp=camera_data["analysis_timestamp"],
            neo_object=camera_data["neo_object"],
            created_at=camera_data["created_at"],
            update_at=camera_data.get("update_at", camera_data["created_at"])
        )
        camera.save()

    print("Dummy data populated successfully.")


In [ ]:
def generate_dummy_zones(num_zones):
    dummy_zones = []
    for i in range(num_zones):
        zone_id = f"zone_{i+1}"
        name = f"Zone {i+1}"
        store_id = "your_store_id" 
        colour_hex = f"#{random.randint(0, 0xFFFFFF):06x}" 
        
        zone_data = {
            'id': zone_id,
            'name': name,
            'store': store_id,
            'colourHex': colour_hex
        }
        dummy_zones.append(zone_data)

    return dummy_zones

def save_dummy_zones(dummy_zones):
    for zone_data in dummy_zones:
        zone = Zone(**zone_data)
        zone.save()

In [ ]:
def populate_videos(num_records):
    for _ in range(num_records):
        # Generate dummy values for each field
        store_id = 'some_store_id'  # Replace with actual store IDs
        camera_id = 'some_camera_id'  # Replace with actual camera IDs
        source_url = fake.url()
        upload_timestamp = fake.date_time_this_year()
        start_time = fake.date_time_this_year()
        end_time = fake.date_time_this_year()
        
        # Create a new Video document with the generated data
        video = Video(
            store_id=store_id,
            camera_id=camera_id,
            source_url=source_url,
            upload_timestamp=upload_timestamp,
            start_time=start_time,
            end_time=end_time
        )
        video.save()


In [ ]:
def main():
    try:
        connect_to_mongo()
        print("Database connection successful..")
        
        n = 10

        dummy_tracking_id(n)
        populate_camera_data()
        dummy_zones = generate_dummy_zones(n)
        save_dummy_zones(dummy_zones)
        populate_videos(n)

        print(f"{n} dummy records inserted into the Tracking_id collection.")
        
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


if __name__ == "__main__":
    main()